In [3]:
pip install boto3


  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/c0/67/4d23a38313d7b37892a6d9c9260809f1a2f5a37feaf6f13da0aa27f57d6d/boto3-1.28.63-py3-none-any.whl.metadata
  Using cached boto3-1.28.63-py3-none-any.whl.metadata (6.7 kB)
  Obtaining dependency information for botocore<1.32.0,>=1.31.63 from https://files.pythonhosted.org/packages/24/0e/39117ec73ea22e700503b3af1dbab270563d6d6ca862cf572899824e4212/botocore-1.31.63-py3-none-any.whl.metadata
  Using cached botocore-1.31.63-py3-none-any.whl.metadata (6.1 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
  Using cached s3transfer-0.7.0-py3-none-any.whl.metadata (1.8 kB)
Using cached boto3-1.28.63-py3-none-any.whl (135 kB)
Using cached botocore-1.31.63-py3-none-any.whl (11.3 MB)
U

## For using External stage as intermediate staging medium 

In [17]:
import cx_Oracle

import boto3,json

import snowflake.connector

 

# Oracle Database Connection

oracle_connection = cx_Oracle.connect('system/admin@localhost:1521/XE')

oracle_cursor = oracle_connection.cursor()

 

#Snowflake Connection

with open('Credentials.json') as config_file:
    config = json.load(config_file)


snowflake_connection = snowflake.connector.connect(

    user = config["USER"],

    password = config["PASSWORD"],

    account = config["ACCOUNT"],

    #warehouse= config["WAREHOUSE"],

    database= config["DATABASE"],

    schema= config["SCHEMA"]

)

 

# Export data from Oracle to CSV

oracle_cursor.execute('select * from employees')

data = oracle_cursor.fetchall()

 

# Write data to a CSV file

with open('data.csv', 'w') as csv_file:

    for row in data:

        csv_file.write(','.join(map(str, row)) + '\n')

 

# Upload CSV to S3

s3_client = boto3.client('s3', aws_access_key_id = config["aws_access_key_id"], aws_secret_access_key = config["aws_access_key_id"])

s3_client.upload_file('employee.csv', 'etlsnowflake21', 'employee.csv')

 

# Loading CSV data from S3 into Snowflake

snowflake_cursor = snowflake_connection.cursor()

STAGE_NAME="@demo_db.public.ext_csv_stage"

snowflake_cursor.execute("COPY INTO DEMO_DB.PUBLIC.EMPLOYEES FROM {} FILE_FORMAT=(TYPE=CSV SKIP_HEADER=0)".format(STAGE_NAME))

 

# Closing my connections

oracle_cursor.close()

oracle_connection.close()

snowflake_cursor.close()

snowflake_connection.close()



DatabaseError: ORA-00922: missing or invalid option

## For using External stage as intermediate staging medium 

In [22]:
import cx_Oracle
import snowflake.connector
import json
 

# Oracle Database Connection

oracle_connection = cx_Oracle.connect('system/admin@localhost:1521/XE')

oracle_cursor = oracle_connection.cursor()

 

with open('Credentials.json') as config_file:
    config = json.load(config_file)


# Snowflake Connection

snowflake_connection = snowflake.connector.connect(

    user = config["USER"],

    password = config["PASSWORD"],

    account = config["ACCOUNT"],

    #warehouse= config["WAREHOUSE"],

    database= config["DATABASE"],

    schema= config["SCHEMA"]

)

 

# Export data from Oracle to CSV

oracle_cursor.execute('SELECT * FROM employees')

data = oracle_cursor.fetchall()

 

# Write data to a CSV file

with open('data.csv', 'w') as csv_file:

    for row in data:

        csv_file.write(','.join(map(str, row)) + '\n')

 

# Define Snowflake stage name

snowflake_stage = "demo_db.public.stg1"

 

# Copy data from the local CSV file to Snowflake stage

snowflake_cursor = snowflake_connection.cursor()

snowflake_cursor.execute(f"PUT file://data.csv @{snowflake_stage}")

 

# Load data from the stage into Snowflake table

snowflake_cursor.execute(f"COPY INTO employees FROM @{snowflake_stage}/data.csv FILE_FORMAT=(TYPE=CSV SKIP_HEADER=0)")

 

# Close database connections

oracle_cursor.close()

oracle_connection.close()

snowflake_cursor.close()

snowflake_connection.close()